In [20]:
import cv2
import imutils
import numpy as np
import pytesseract
# to compute angles
from math import atan2, cos, sin, sqrt, pi
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from skimage.filters import threshold_local
import tensorflow as tf
from skimage import measure

In [21]:
def clean_plate(self, plate):
 
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
     
    contours, _ = cv2.findContours(thresh.copy(),
                                      cv2.RETR_EXTERNAL,
                                      cv2.CHAIN_APPROX_NONE)
 
    if contours:
         
        areas = [cv2.contourArea(c) for c in contours]
         
        # index of the largest contour in the
        # areas array
        max_index = np.argmax(areas) 
 
        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        x, y, w, h = cv2.boundingRect(max_cnt)
 
        if not self.ratioCheck(max_cntArea,
                               plate.shape[1],
                               plate.shape[0]):
            return plate, False, None
         
        return plate, True, [x, y, w, h]
     
    else:
        return plate, False, None
    
def ratioCheck(self, area, width, height):
     
    min = self.min_area
    max = self.max_area
 
    ratioMin = 3
    ratioMax = 6
 
    ratio = float(width) / float(height)
     
    if ratio < 1:
        ratio = 1 / ratio
 
    if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
        return False
     
    return True

In [25]:
def sort_cont(character_contours):
	"""
	To sort contours
	"""
	i = 0
	boundingBoxes = [cv2.boundingRect(c) for c in character_contours]
	
	(character_contours, boundingBoxes) = zip(*sorted(zip(character_contours,
														boundingBoxes),
													key = lambda b: b[1][i],
													reverse = False))
	
	return character_contours


def segment_chars(plate_img, fixed_width):
	
	"""
	extract Value channel from the HSV format
	of image and apply adaptive thresholding
	to reveal the characters on the license plate
	"""
	V = cv2.split(cv2.cvtColor(plate_img, cv2.COLOR_BGR2HSV))[2]

	thresh = cv2.adaptiveThreshold(V, 255,
								cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
								cv2.THRESH_BINARY,
								11, 2)

	thresh = cv2.bitwise_not(thresh)

	# resize the license plate region to
	# a canoncial size
	plate_img = imutils.resize(plate_img, width = fixed_width)
	thresh = imutils.resize(thresh, width = fixed_width)
	bgr_thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

	# perform a connected components analysis
	# and initialize the mask to store the locations
	# of the character candidates
	labels = measure.label(thresh, background = 0)

	charCandidates = np.zeros(thresh.shape, dtype ='uint8')

	# loop over the unique components
	characters = []
	for label in np.unique(labels):
		
		# if this is the background label, ignore it
		if label == 0:
			continue
		# otherwise, construct the label mask to display
		# only connected components for the current label,
		# then find contours in the label mask
		labelMask = np.zeros(thresh.shape, dtype ='uint8')
		labelMask[labels == label] = 255

		cnts = cv2.findContours(labelMask,
					cv2.RETR_EXTERNAL,
					cv2.CHAIN_APPROX_SIMPLE)

		cnts = cnts[0]

		# ensure at least one contour was found in the mask
		if len(cnts) > 0:

			# grab the largest contour which corresponds
			# to the component in the mask, then grab the
			# bounding box for the contour
			c = max(cnts, key = cv2.contourArea)
			(boxX, boxY, boxW, boxH) = cv2.boundingRect(c)

			# compute the aspect ratio, solodity, and
			# height ration for the component
			aspectRatio = boxW / float(boxH)
			solidity = cv2.contourArea(c) / float(boxW * boxH)
			heightRatio = boxH / float(plate_img.shape[0])

			# determine if the aspect ratio, solidity,
			# and height of the contour pass the rules
			# tests
			keepAspectRatio = aspectRatio < 1.0
			keepSolidity = solidity > 0.15
			keepHeight = heightRatio > 0.5 and heightRatio < 0.95

			# check to see if the component passes
			# all the tests
			if keepAspectRatio and keepSolidity and keepHeight and boxW > 14:
				
				# compute the convex hull of the contour
				# and draw it on the character candidates
				# mask
				hull = cv2.convexHull(c)

				cv2.drawContours(charCandidates, [hull], -1, 255, -1)

	contours, hier = cv2.findContours(charCandidates,
										cv2.RETR_EXTERNAL,
										cv2.CHAIN_APPROX_SIMPLE)
	
	if contours:
		contours = sort_cont(contours)
		
		# value to be added to each dimension
		# of the character
		addPixel = 4
		for c in contours:
			(x, y, w, h) = cv2.boundingRect(c)
			if y > addPixel:
				y = y - addPixel
			else:
				y = 0
			if x > addPixel:
				x = x - addPixel
			else:
				x = 0
			temp = bgr_thresh[y:y + h + (addPixel * 2),
							x:x + w + (addPixel * 2)]

			characters.append(temp)
			
		return characters
	
	else:
		return None

class PlateFinder:
	def __init__(self):
		
		# minimum area of the plate
		self.min_area = 4500
		
		# maximum area of the plate
		self.max_area = 30000

		self.element_structure = cv2.getStructuringElement(
							shape = cv2.MORPH_RECT, ksize =(22, 3))

	def preprocess(self, input_img):
		
		imgBlurred = cv2.GaussianBlur(input_img, (7, 7), 0)
		
		# convert to gray
		gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)
		
		# sobelX to get the vertical edges
		sobelx = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
		
		# otsu's thresholding
		ret2, threshold_img = cv2.threshold(sobelx, 0, 255,
						cv2.THRESH_BINARY + cv2.THRESH_OTSU)

		element = self.element_structure
		morph_n_thresholded_img = threshold_img.copy()
		cv2.morphologyEx(src = threshold_img,
						op = cv2.MORPH_CLOSE,
						kernel = element,
						dst = morph_n_thresholded_img)
		
		return morph_n_thresholded_img

	def extract_contours(self, after_preprocess):
		
		contours, _ = cv2.findContours(after_preprocess,
										mode = cv2.RETR_EXTERNAL,
										method = cv2.CHAIN_APPROX_NONE)
		return contours

	def clean_plate(self, plate):
		
		gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
		thresh = cv2.adaptiveThreshold(gray,
									255,
									cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
									cv2.THRESH_BINARY,
									11, 2)
		
		contours, _ = cv2.findContours(thresh.copy(),
										cv2.RETR_EXTERNAL,
										cv2.CHAIN_APPROX_NONE)

		if contours:
			areas = [cv2.contourArea(c) for c in contours]
			
			# index of the largest contour in the area
			# array
			max_index = np.argmax(areas)

			max_cnt = contours[max_index]
			max_cntArea = areas[max_index]
			x, y, w, h = cv2.boundingRect(max_cnt)
			rect = cv2.minAreaRect(max_cnt)
			
			if not self.ratioCheck(max_cntArea, plate.shape[1],
												plate.shape[0]):
				return plate, False, None
			
			return plate, True, [x, y, w, h]
		
		else:
			return plate, False, None



	def check_plate(self, input_img, contour):
		
		min_rect = cv2.minAreaRect(contour)
		
		if self.validateRatio(min_rect):
			x, y, w, h = cv2.boundingRect(contour)
			after_validation_img = input_img[y:y + h, x:x + w]
			after_clean_plate_img, plateFound, coordinates = self.clean_plate(
														after_validation_img)
			
			if plateFound:
				characters_on_plate = self.find_characters_on_plate(
											after_clean_plate_img)
				
				if (characters_on_plate is not None and len(characters_on_plate) == 8):
					x1, y1, w1, h1 = coordinates
					coordinates = x1 + x, y1 + y
					after_check_plate_img = after_clean_plate_img
					
					return after_check_plate_img, characters_on_plate, coordinates
		
		return None, None, None

	def find_possible_plates(self, input_img):
		
		"""
		Finding all possible contours that can be plates
		"""
		plates = []
		self.char_on_plate = []
		self.corresponding_area = []

		self.after_preprocess = self.preprocess(input_img)
		possible_plate_contours = self.extract_contours(self.after_preprocess)

		for cnts in possible_plate_contours:
			plate, characters_on_plate, coordinates = self.check_plate(input_img, cnts)
			
			if plate is not None:
				plates.append(plate)
				self.char_on_plate.append(characters_on_plate)
				self.corresponding_area.append(coordinates)

		if (len(plates) > 0):
			return plates
		
		else:
			return None

	def find_characters_on_plate(self, plate):

		charactersFound = segment_chars(plate, 400)
		if charactersFound:
			return charactersFound

	# PLATE FEATURES
	def ratioCheck(self, area, width, height):
		
		min = self.min_area
		max = self.max_area

		ratioMin = 3
		ratioMax = 6

		ratio = float(width) / float(height)
		
		if ratio < 1:
			ratio = 1 / ratio
		
		if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
			return False
		
		return True

	def preRatioCheck(self, area, width, height):
		
		min = self.min_area
		max = self.max_area

		ratioMin = 2.5
		ratioMax = 7

		ratio = float(width) / float(height)
		
		if ratio < 1:
			ratio = 1 / ratio

		if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
			return False
		
		return True

	def validateRatio(self, rect):
		(x, y), (width, height), rect_angle = rect

		if (width > height):
			angle = -rect_angle
		else:
			angle = 90 + rect_angle

		if angle > 15:
			return False
		
		if (height == 0 or width == 0):
			return False

		area = width * height
		
		if not self.preRatioCheck(area, width, height):
			return False
		else:
			return True


In [26]:
class OCR:
     
    def __init__(self):
         
        self.model_file = "model/binary_128_0.50_ver3.pb"
        self.label_file = "model/binary_128_0.50_labels_ver2.txt"
        self.label = self.load_label(self.label_file)
        self.graph = self.load_graph(self.model_file)
        self.sess = tf.compat.v1.Session(graph = self.graph)
 
    def load_graph(self, modelFile):
         
        graph = tf.Graph()
        graph_def = tf.compat.v1.GraphDef()
         
        with open(modelFile, "rb") as f:
            graph_def.ParseFromString(f.read())
         
        with graph.as_default():
            tf.import_graph_def(graph_def)
         
        return graph
 
    def load_label(self, labelFile):
        label = []
        proto_as_ascii_lines = tf.io.gfile.GFile(labelFile).readlines()
         
        for l in proto_as_ascii_lines:
            label.append(l.rstrip())
         
        return label
 
    def convert_tensor(self, image, imageSizeOuput):
        """
        takes an image and transform it in tensor
        """
        image = cv2.resize(image,
                           dsize =(imageSizeOuput,
                                  imageSizeOuput),
                           interpolation = cv2.INTER_CUBIC)
         
        np_image_data = np.asarray(image)
        np_image_data = cv2.normalize(np_image_data.astype('float'),
                                      None, -0.5, .5,
                                      cv2.NORM_MINMAX)
         
        np_final = np.expand_dims(np_image_data, axis = 0)
         
        return np_final
 
    def label_image(self, tensor):
 
        input_name = "import/input"
        output_name = "import/final_result"
 
        input_operation = self.graph.get_operation_by_name(input_name)
        output_operation = self.graph.get_operation_by_name(output_name)
 
        results = self.sess.run(output_operation.outputs[0],
                                {input_operation.outputs[0]: tensor})
        results = np.squeeze(results)
        labels = self.label
        top = results.argsort()[-1:][::-1]
         
        return labels[top[0]]
 
    def label_image_list(self, listImages, imageSizeOuput):
        plate = ""
         
        for img in listImages:
             
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            plate = plate + self.label_image(self.convert_tensor(img, imageSizeOuput))
         
        return plate, len(plate)

In [27]:
findPlate = PlateFinder()
model = OCR()

cap = cv2.VideoCapture('video.MOV')

while (cap.isOpened()):
    ret, img = cap.read()

    if ret == True:
        cv2.imshow('original video', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        possible_plates = findPlate.find_possible_plates(img)

        if possible_plates is not None:

            for i, p in enumerate(possible_plates):
                chars_on_plate = findPlate.char_on_plate[i]
                recognized_plate, _ = model.label_image_list(
                           chars_on_plate, imageSizeOuput = 128)

                print(recognized_plate)
                cv2.imshow('plate', p)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
    else:
        break

29A39185
29A33185
29A33185
29A39185


KeyboardInterrupt: 

In [19]:

for i in range(1, 20):

    img    = cv2.imread(f"archive/images/Cars{i}.png")
    # cv2.imshow('HelloWorld', img)
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(grey, 13, 15, 15) 

    edged = cv2.Canny(gray, 30, 200) 
    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None


    for c in contours:
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is None:
        detected = 0
        print ("No contour detected")
        continue
        
    else:
        detected = 1

    if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)

No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected


In [20]:
import time
import math

cap = cv2.VideoCapture('Traffic_Camera.avi')

frameRate = cap.get(5) #frame rate

x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = './test_images/image' +  str(int(x)) + ".jpg";x+=1
        cv2.imwrite(filename, frame)
cap.release()

In [21]:
for i in range(1, 30):

    img = cv2.imread(f"test_images/image{i}.jpg")
    y=0
    x=1400
    h=700
    w=500
    img = img[y:y+h, x:x+w]
#     cv2.imshow('car',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(grey, 13, 15, 15) 

    edged = cv2.Canny(gray, 30, 200) 
    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None

    for c in contours:
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is None:
        detected = 0
        print ("No contour detected")
        continue
        
    else:
        detected = 1

    if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)
    (x, y) = np.where(mask == 255)
    (topx, topy) = (np.min(x), np.min(y))
    (bottomx, bottomy) = (np.max(x), np.max(y))
    Cropped = gray[topx:bottomx+1, topy:bottomy+1]

    text = pytesseract.image_to_string(Cropped)
    print("programming_fever's License Plate Recognition\n")
    print("Detected license plate Number is:",text)
    img = cv2.resize(img,(500,700))
    Cropped = cv2.resize(Cropped,(400,200))
#     cv2.imshow('car',img)
#     cv2.imshow('Cropped',Cropped)

#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    

programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
prog

In [22]:
def draw_axis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    
def get_orientation(pts, img):
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 *  eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    draw_axis(img, cntr, p1, (0, 150, 0), 1)
    draw_axis(img, cntr, p2, (200, 150, 0), 5)
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    return angle

In [26]:
for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break


if screenCnt is None:
    detected = 0
    print ("No contour detected")
    
    
else:
    detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

get_orientation(c,img)
cv2.imshow('frame',img)
cv2.waitKey(0)
cv2.destroyAllWindows()